In [7]:
# Author: Turevskyi D.V.
# Model type: Neural network (Computer Vision)
# Description: Classification of butterfly (100 type) 

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision.models import ResNet50_Weights 

In [8]:
# Data preparation and parameter setting
batch_size = 64
img_size = 224

num_epochs = 10
device = torch.device('cuda')

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(root='dataset/butterfly/train/', transform=transform)
valid_dataset = datasets.ImageFolder(root='dataset/butterfly/valid/', transform=transform)
test_dataset = datasets.ImageFolder(root='dataset/butterfly/test/', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=6, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=6, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=6, shuffle=False)

In [9]:
# ResNet50 pre-trained model
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Replace the last layer with ours with 100 classes
model.fc = nn.Linear(model.fc.in_features, 100)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

best_accuracy = 0.0
patience_counter = 0
patience_limit = 5 

# Learning cycle
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    scheduler.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    # Validation
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Validation Loss: {valid_loss/len(valid_loader)}, Accuracy: {accuracy}%')
    
    # Save the model if the accuracy has improved
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'butterfly_classifier_best.pth')
        print("Saved best model with accuracy:", best_accuracy)
        patience_counter = 0  
    else:
        patience_counter += 1  
    
    if patience_counter >= patience_limit:
        print("Early stopping triggered")
        break

Epoch [1/10], Loss: 1.8530569896480154
Validation Loss: 0.3180398242548108, Accuracy: 91.2%
Saved best model with accuracy: 91.2
Epoch [2/10], Loss: 0.2626589573730672
Validation Loss: 0.21125379018485546, Accuracy: 95.0%
Saved best model with accuracy: 95.0
Epoch [3/10], Loss: 0.14510958866797727
Validation Loss: 0.2060359437018633, Accuracy: 95.6%
Saved best model with accuracy: 95.6
Epoch [4/10], Loss: 0.09967803077649344
Validation Loss: 0.1932400008663535, Accuracy: 95.2%
Epoch [5/10], Loss: 0.0720548183955063
Validation Loss: 0.1822917303070426, Accuracy: 95.4%
Epoch [6/10], Loss: 0.05366475118068874
Validation Loss: 0.1972350999712944, Accuracy: 96.2%
Saved best model with accuracy: 96.2
Epoch [7/10], Loss: 0.04373754452450702
Validation Loss: 0.20878932997584343, Accuracy: 95.2%
Epoch [8/10], Loss: 0.024712240313969288
Validation Loss: 0.16766766575165093, Accuracy: 96.4%
Saved best model with accuracy: 96.4
Epoch [9/10], Loss: 0.017421004740606406
Validation Loss: 0.1716393153

In [10]:
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Loss: {test_loss/len(test_loader)}, Test Accuracy: {100 * correct / total}%')


Test Loss: 0.07383105653570965, Test Accuracy: 98.0%
